First, implement a version of the naive exact matching algorithm that is strand-aware. That is, instead of looking only for occurrences of P in T, additionally look for occurrences of thereverse complement of P in T. If P is ACT, your function should find occurrences of both ACTand its reverse complement AGT in T.

If P and its reverse complement are identical (e.g. AACGTT), then a given match offset should be reported only once. So if your new function is called naive_with_rc, then the old naivefunction and your new naive_with_rc function should return the same results when P equals its reverse complement.

Next, download and parse the lambda virus genome, <a href="https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/lambda_virus.fa">here</a>.

In [1]:
#234567890#234567890#234567890#234567890#234567890#234567890#234567890#234567890
DATA = '../data'

In [2]:
!ls $DATA

dna.example.fasta           phix.fa
ERR037900_1.first1000.fastq SRR835775_1.first1000.fastq
lambda_virus.fa


In [3]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

In [4]:
def reverse_complement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [5]:
def naive_with_rc(p, t):
    # look for occurrences of p in t, and for occurences of rc(p) in t
    # return indices for each match
    occurrences = []
    matches = naive(p, t)
    occurrences += matches
    rc = reverse_complement(p)
    if rc != p:
        rc_matches = naive(rc, t)
        occurrences += rc_matches
    return sorted(occurrences)

In [6]:
# Ex 1
p = 'CCC'
ten_as = 'AAAAAAAAAA'
t = ten_as + 'CCC' + ten_as + 'GGG' + ten_as
occurrences = naive_with_rc(p, t)
print(p, t)
print(occurrences)   # 10, 23

CCC AAAAAAAAAACCCAAAAAAAAAAGGGAAAAAAAAAA
[10, 23]


In [7]:
p = 'CGCG'
t = ten_as + 'CGCG' + ten_as + 'CGCG' + ten_as
occurrences = naive_with_rc(p, t)
print(occurrences)  # 10, 24

[10, 24]


In [8]:
def read_genome(path):
    genome = ''
    with open(path, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [9]:
phix_genome = read_genome(f'{DATA}/phix.fa')

In [10]:
phix_genome[:100]

'GAGTTTTATCGCTTCCATGACGCAGAAGTTAACACTTTCGGATATTTCTGATGAGTCGAAAAATTATCTTGATAAAGCAGGAATTACTACTGCTTGTTTA'

In [11]:
occurrences = naive_with_rc('ATTA', phix_genome)

In [12]:
print('offset of leftmost occurrence:', min(occurrences))  # 62
print('N occurrences:', len(occurrences))  # 60

offset of leftmost occurrence: 62
N occurrences: 60


In [13]:
def read_fastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [14]:
t = read_genome(f'{DATA}/lambda_virus.fa')
t[:100]

'GGGCGGCGACCTCGCGGGTTTTCGCTATTTATGAAAATTTTCCGGTTTAAGGCGTTTCCGTTCTTCTTCGTCATAACTTAATGTTTTTATTTAAAATACC'

In [15]:
# 1.
p = 'AGGT'
matches = naive_with_rc(p, t)
len(matches)

306

In [16]:
# 2.
p = 'TTAA'
matches = naive_with_rc(p, t)
len(matches)

195

In [17]:
matches = naive(p, t)
len(matches)

195

In [18]:
# 3.
p = 'ACTAAGT'
matches = naive_with_rc(p, t)
matches[0]

26028

In [19]:
t[26028:26035]

'ACTTAGT'

In [20]:
# 4.
p = 'AGTCGA'
matches = naive_with_rc(p, t)
matches[0]

450

In [21]:
t[450:456]

'TCGACT'

In [22]:
def naive_mm(p, t, mm=2):
    occurrences = []
    mms = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                mms += 1
                if mms > mm:
                    match = False
                    break
        if match:
            occurrences.append(i)  # all chars matched; record
        mms = 0
    return occurrences

In [23]:
p = 'ACTTTA'
t = 'ACTTACTTGATAAAGT'
naive_mm(p, t)

[0, 4]

In [24]:
# 5. 
t = read_genome(f'{DATA}/lambda_virus.fa')
p = 'TTCAAGCC'
matches = naive_mm(p, t)
len(matches)

191

In [25]:
# 6.
p = 'AGGAGGTT'
matches = naive_mm(p, t)
matches[0]

49

In [59]:
# 7.
err = read_fastq(f'{DATA}/ERR037900_1.first1000.fastq')
len(err)

2

In [60]:
seq, conf = err

In [61]:
seq[0]

'TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCNAACCCTAACCCTAACCCTAACCCTAACCCTAAC'

In [62]:
conf[0]

'HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHGFHHHFHFFHHHHHGHHFHEH@4#55554455HGFBF<@C>7EEF@FBEDDD<=C<E'

In [63]:
len(seq[0])

100

In [65]:
seq[:10]

['TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCNAACCCTAACCCTAACCCTAACCCTAACCCTAAC',
 'TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCNAACCCTAACCCTAACCCTAACCCTNACCCTAAC',
 'TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCNAACCCTAACCCTAACCCTAACCCTAACCCTAAC',
 'TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCNAACCCTAACCCTAACCCTAACCCTAACCCTACC',
 'AACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTNACCCTAACCCTAACCCTAACCCTAAACCTAACC',
 'AACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTNACCCTAACCCTAACCCTAACCCTAACCCTAACC',
 'AACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTNACCCTAACCCTAACCCTAACCCTAACCCTAACC',
 'AACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTNAACCTAACCCTAACCCTAACTCTAACCCTAACC',
 'ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTANCCCTAACCCTAACCCTAACCCTAACCCTAACCC',
 'CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTA

In [66]:
seq[0].find('N')

66